### Purpose

The intent of this notebook is to serve as a rapid testing ground for new utilities. Any logic written here should migrate to the src/ directory as proper functions.

#### Import and Constants

In [ ]:
%load_ext autoreload
from datetime import datetime, timedelta
import pandas as pd
import numpy as np
from pandas.core.generic import NDFrame

# Importing from the local code structure
import os
import sys
module_path = os.path.abspath(os.path.join("..", "src"))
sys.path.append(module_path)
from exercise_log.dataloader import DataLoader, ColumnName as CName
from exercise_log.strength import CardioType, Exercise, SetRating
from exercise_log.trend import BikeCardioSummary, FootCardioSummary, WeightTrainingSummary, Trendsetter
from exercise_log.utils import TermColour, convert_mins_to_hour_mins
from exercise_log.vis import plot_resting_heart_rate, plot_strength_over_time, plot_weight, plot_workout_frequency

%autoreload

#### Load Data and Compute Trends

In [ ]:
EXTRAPOLATE_DAYS = 150
N_DAYS_TO_AVG = 28
ROOT_DATA_DIR = "../../data"
ROOT_IMG_DIR = "../../img"

health_metrics = DataLoader.load_health_metrics(ROOT_DATA_DIR)
travel_days = DataLoader.load_travel_days(ROOT_DATA_DIR)
walks = DataLoader.load_walk_workouts(ROOT_DATA_DIR)
runs = DataLoader.load_run_workouts(ROOT_DATA_DIR)
bikes = DataLoader.load_bike_workouts(ROOT_DATA_DIR)
rows = DataLoader.load_row_workouts(ROOT_DATA_DIR)
stairs = DataLoader.load_stair_workouts(ROOT_DATA_DIR)
cardio_workouts = DataLoader.merge_cardio_workouts([
        walks,
        runs,
        bikes,
        rows,
        stairs,
    ])
weight_training_workouts = DataLoader.load_weight_training_workouts(ROOT_DATA_DIR)
weight_training_sets = DataLoader.load_weight_training_sets(ROOT_DATA_DIR)
rate_of_climb = DataLoader.load_rate_of_climb(ROOT_DATA_DIR)
dashes = DataLoader.load_dashes(ROOT_DATA_DIR)
walk_backwards = DataLoader.load_walk_backwards(ROOT_DATA_DIR)
all_workouts = DataLoader.load_all_workouts(cardio_workouts, weight_training_workouts, travel_days)

# n-day average gives a sense of if I'm keeping above the recommended minimum exercise duration of 150 minutes/week
n_day_avg_workout_duration = Trendsetter.compute_n_sample_avg(all_workouts, CName.DURATION, N_DAYS_TO_AVG)

# Fit relevant trendlines
weight_trendline = Trendsetter.get_line_of_best_fit(health_metrics, CName.WEIGHT, EXTRAPOLATE_DAYS)
heart_rate_trendline = Trendsetter.get_logarithmic_curve_of_best_fit(health_metrics, CName.RESTING_HEART_RATE, EXTRAPOLATE_DAYS)

In [ ]:
nonnulls = health_metrics[health_metrics[CName.WEIGHT].notnull()]
m, b = Trendsetter.fit_linear(nonnulls, CName.WEIGHT)
print("Average daily change in weight: {:.2f}lb".format(m))

expected = m * health_metrics.index + b
squared_error = (expected - health_metrics[CName.WEIGHT]) ** 2
avg_weight = health_metrics[CName.WEIGHT].mean()
squares = (health_metrics[CName.WEIGHT] - avg_weight) ** 2
r_squared = 1 - (squared_error.sum() / squares.sum())
print("r^2 value of line of best fit: {:.3f}".format(r_squared))

daily_changes = health_metrics[CName.WEIGHT].diff()
max_gain, max_loss = round(daily_changes.max(), 1), round(daily_changes.min(), 1)
print(f"Max day-to-day loss/gain: {max_loss}lb/{max_gain}lb")

#### Recent Cardio Workouts

In [ ]:
%%html
<style>.dataframe th { font-size: 10px; } .dataframe td { font-size: 10px; }</style>

In [ ]:
DataLoader.add_computed_cardio_metrics(walks)
DataLoader.add_computed_cardio_metrics(runs)
DataLoader.add_computed_cardio_metrics(bikes)
DataLoader.add_computed_cardio_metrics(rows)
DataLoader.add_computed_cardio_metrics(rate_of_climb)
DataLoader.add_computed_cardio_metrics(dashes)
DataLoader.add_computed_cardio_metrics(walk_backwards)

# Add best results for each column
def add_column_maxes(df: pd.DataFrame):
    column_maxes = pd.DataFrame({column: [np.nan] for column in df.columns})
    column_maxes[CName.WORKOUT_TYPE] = "Max Values"
    for c_name in CName:
        if c_name in {CName.WORKOUT_TYPE}:
            continue
        if c_name not in df:
            continue
        column_maxes[c_name] = df[c_name].max()
    column_maxes[CName.NOTES] = ""
    column_maxes[CName.DATE] = ""
    return pd.concat([df, column_maxes], ignore_index=True)
walks_with_max =  add_column_maxes(walks)
runs_with_max =  add_column_maxes(runs)
bikes_with_max =  add_column_maxes(bikes)
rows_with_max =  add_column_maxes(rows)

display(walks[-10:])
display(runs[-10:])
display(bikes[-10:])
display(rows[-10:])

#### Build Visuals

In [ ]:
# TODO Still need to build at least these visuals:
# * Walking data (max distance, max elevation gain, max duration, pace graph)
# * Strength Metrics grouped by workout
#    - Ideally: drop-down menu to select between various workouts
#    - Need to fine-tune some, remove ones that don't make sense (e.g. that have < 3 days worked)
from exercise_log.run_updater import HealthTrends
health_trends = HealthTrends(all_workouts, health_metrics, EXTRAPOLATE_DAYS)
plot_workout_frequency(health_trends.get_workout_durations(), N_DAYS_TO_AVG)
plot_resting_heart_rate(health_trends.health_metrics, health_trends.get_heart_rate_trendline())
plot_weight(health_trends.health_metrics, health_trends.get_weight_trendline())

# TODO [Workout frequency graph]
# Change colour of rest days to red or orange and MAYBE also color-code cardio vs weights
# Another idea would be generating sub-graphs for each step of the hierarchy. E.g.
# All -> Cardio -> Walk
# All -> Weight -> Chest (per muscle group, not pairs, bc I don't want to marry the visuals to the current splits)

#### Misc. Metrics

In [ ]:
# Construct a full-picture set of functional fitness metrics
# Will also need to work out a decent way to track progress across so many different metrics, e.g.:
#   - compute and graph aggregate metrics over time
#   - interface that allows drilling-in per-metric and/or group of metrics (e.g. arm strength, walking metrics, etc.)
# Include: 
#   -  Major strength metrics: 1RM deadlift, squat, bench press, strict press
#   2. Cardio metrics:
#        Walking: 1km time, 5km time, 1000ft climb time, max 1-min walking pace
#        Running: 100m sprint, 1 mile time, 5km time, half-marathon time, marathon time, maximum duration
#        Swimming: 400m freestyle time?, 1000m freestyle time?
#        Cardio Health: 2-week average resting heart rate, V02 Max
#   3. Calisthenic metrics: max consecutive push-ups, pull-ups, burpees; static hang duration, vertical leap?
#   4. Minor strength metrics:
#        Chest: N/A
#        Arms: 1RM overhead tricep extension, preacher curl
#        Legs: 1RM leg curl, leg extension, seated calf raise
#        Back: 1RM lawnmowers, lat pulldown
#        Shoulder: 1RM arnold press
#        Forearms: grip strength
#   5. Imbalance measures:
#        Single-arm/leg bicep curl, overhead tricep extension, leg curl, leg extension, etc. vs two-arm/leg
#          Ideally two-arm/leg should be within a few % of 2 * single-arm/leg
#        Relative strength scores
#          within region (e.g. tricep vs. bicep, quads vs. hamstring, back vs. chest)
#          global (e.g. detect outliers such as "weak triceps", compute some total imbalance score)
#   6. Detailed Training Frequency Info
#        Percentage + absolute volume/time of {weight training, calisthenics, cardio}
#        Optionally: include a drill down on compound lifts, isolated lifts, muscle groups, cardio types
#        Optionally: include prescriptive piece to offer a few recommendations for the next workout (e.g. cardio of type X or weight training of type X with a focus on {low, mid, high} reps)
from exercise_log.constants import number, Union
def get_1rm(sets: NDFrame, exercise: str) -> Union[number, str]:
    sets = sets[sets[CName.EXERCISE] == exercise]
    sets = sets[sets[CName.REPS] == 1]
    sets = sets[sets[CName.RATING] == SetRating.GOOD]
    if sets.empty:
        return "Missing Data"
    return sets[CName.WEIGHT].max()

# Functional Fitness Metric Group 1: Big-Three 1-RM
# TODO modify 1RM to only check sets from the past N-months (3 months maybe?)
one_rm = dict()
for exercise in [Exercise.DEADLIFT, Exercise.SQUATS, Exercise.BENCH_PRESS, Exercise.STRICT_PRESS]:
    one_rm[exercise] = get_1rm(weight_training_sets, exercise)
    print(f"1-RM {exercise}: {int(one_rm[exercise])}")
print(f'Big-Three Combined Weight: {int(one_rm[Exercise.DEADLIFT] + one_rm[Exercise.SQUATS] + one_rm[Exercise.BENCH_PRESS])}')

# Functional Fitness Metric Group 4: Minor Strength Metrics
# TODO same two TODO's as Group 1
print()
for exercise in [
    Exercise.OVERHEAD_TRICEP_EXTENSION, Exercise.PREACHER_CURL,
    Exercise.SINGLE_LEG_LEG_CURL, Exercise.SINGLE_LEG_LEG_EXTENSION, Exercise.BARBELL_CALF_RAISE,
    Exercise.LAWNMOWERS, Exercise.LAT_PULLDOWN,
    Exercise.ARNOLD_PRESS,
]:
    one_rm[exercise] = get_1rm(weight_training_sets, exercise)
    if one_rm[exercise] != "Missing Data":
        one_rm[exercise] = int(one_rm[exercise])
    print(f"1-RM {exercise}: {one_rm[exercise]}")


distance_walked = round(walks[CName.DISTANCE].to_numpy().sum())
farthest = walks[CName.DISTANCE].to_numpy().max()
fastest_pace = walks[CName.PACE].to_numpy().max()
print()
print("Walking Metrics")
print(f"Farthest distance: {farthest}km")
print("Fastest pace: {:.2f}m/s".format(fastest_pace))
print(f"Total distance: {distance_walked:,}km")

distance_ran = round(runs[CName.DISTANCE].to_numpy().sum())
farthest = runs[CName.DISTANCE].to_numpy().max()
fastest_pace = runs[CName.PACE].to_numpy().max()
print()
print("Running Metrics")
print(f"Farthest distance: {farthest}km")
print(f"Fastest sprint {dashes[CName.SPEED].max()}kph")
print("Fastest pace: {:.2f}m/s".format(fastest_pace))
print(f"Total distance: {distance_ran:,}km")

nonnulls = bikes[bikes[CName.DISTANCE].notnull()]
distance_biked = round(nonnulls[CName.DISTANCE].to_numpy().sum())
print()
print("Biking Metrics")
print(f"Total distance: {distance_biked:,}km")

distance_travelled = distance_walked + distance_ran + distance_biked
distance_climbed = walks[CName.ELEVATION].to_numpy().sum() + runs[CName.ELEVATION].to_numpy().sum()
print()
print("Summary Metrics")
print(f"Total distance travelled: {distance_travelled:,}km")
print(f"Total elevation gain: {distance_climbed:,} m")

In [ ]:
DAYS_IN_WEEK = 7
SAT_OFFSET = 6
def get_last_week_date_range():
    today = datetime.today().date()
    today_idx = today.weekday() + 1 # MON = 0, SUN = 6 -> SUN = 0 .. SAT = 6
    last_saturday = today - timedelta(today_idx + 1)
    last_sunday = today - timedelta(DAYS_IN_WEEK + today_idx)
    return last_sunday, last_saturday

def get_first_year_date_range(all_workouts: pd.DataFrame):
    first_day = all_workouts[CName.DATE].min().date()
    # This doesn't work for 2/29 of leap years but that's irrelevant here
    end_of_year_one = first_day.replace(year=first_day.year + 1)
    return first_day, end_of_year_one

def print_summaries_for_range(start, end):
    print(f"From {start} to {end} I exercised by...")
    summaries = [
        WeightTrainingSummary.build_summary(weight_training_sets, start, end),
        FootCardioSummary.build_summary(pd.concat([walks, runs]), start, end),
        BikeCardioSummary.build_summary(bikes, start, end),
    ]
    for summary in summaries:
        print(summary)

print_summaries_for_range(*get_last_week_date_range())
print()
print_summaries_for_range(*get_first_year_date_range(all_workouts))


In [ ]:
sessions = []
for i in range(1, 8):
    day = datetime.today().date() - timedelta(i)
    day = np.datetime64(day, "ns")
    sessions.append(weight_training_sets[weight_training_sets[CName.DATE] == day])
sessions

In [ ]:
import json
from exercise_log.strength.ontology import ExerciseInfo

muscle_groups_worked = {}
muscles_worked = {}
weekly_exercises = (exercise for session in sessions for exercise in session[CName.EXERCISE])
for exercise in weekly_exercises:
    exercise_info = ExerciseInfo(exercise)
    for muscle_group in exercise_info.muscle_groups_worked:
        muscle_groups_worked[muscle_group] = muscle_groups_worked.get(muscle_group, 0) + 1
    for muscle in exercise_info.muscles_worked:
        muscles_worked[muscle] = muscles_worked.get(muscle, 0) + 1
print(json.dumps(muscle_groups_worked, sort_keys=True, indent=2))
print(json.dumps(muscles_worked, sort_keys=True, indent=2))